# Домашняя работа

1. Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
2. Построить случайный лес, вывести важность признаков
3. Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
4. В качестве решения: Jupyter notebook с кодом, комментариями и графиками

In [100]:
import pandas as pd
import numpy as np
import matplotlib as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [101]:
hpart_train = pd.read_csv('train.csv')
hpart_test = pd.read_csv('test.csv')

In [102]:
print(hpart_train.shape)
print(hpart_test.shape)

(1460, 81)
(1459, 80)


In [103]:
hpart_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [104]:
hpart_train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


Очень интересная задача:) :У нас есть обучающая и тестовая выборка. И она отличается одним столбцом (column from columns). Это столбец 'SalePrice' - целевая переменная!

### Разделим тренировочную выборку

In [105]:
from sklearn.model_selection import train_test_split
X = hpart_train.copy()
X.drop(columns=['SalePrice'], inplace=True)
y = pd.DataFrame(hpart_train['SalePrice'])

In [106]:
X.shape, y.shape

((1460, 80), (1460, 1))

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [108]:
X_train.shape, y_train.shape

((1168, 80), (1168, 1))

In [109]:
X_test.shape, y_test.shape

((292, 80), (292, 1))

### Предобработка данных

Данных много, есть числовые и не числовые. Но числовых тоже достаточно. Оставим только колонки с числовыми данными. (Не числовые данные можно превратить в дальнейшем в числовые с помощью LabelEncoder или OneHotEncoder). 

In [110]:
X_train_num = X_train.select_dtypes(exclude='object')

In [154]:
X_test_num = X_test.select_dtypes(exclude='object')

Удаляем несущественные признаки.

In [111]:
drop_it = ['Id', 'MSSubClass', 'MoSold']
X_train_num.drop(columns=drop_it, inplace=True)

In [155]:
X_test_num.drop(columns=drop_it, inplace=True)

In [112]:
X_train_num.dtypes.value_counts()

int64      31
float64     3
dtype: int64

In [156]:
X_test_num.dtypes.value_counts()

int64      31
float64     3
dtype: int64

Очищаем данные от пропусков, заменяем пропуски на средние значения.

In [113]:
y_train.isnull().any().any() #Всё ок.

False

In [157]:
y_test.isnull().any().any()

False

In [114]:
X_train_num.isnull().any().any() #Надо заполнить.

True

In [158]:
X_test_num.isnull().any().any()

True

In [115]:
X_train_nnn = X_train_num.fillna(X_train_num.mean())

In [159]:
X_test_nnn = X_test_num.fillna(X_test_num.mean())

In [160]:
X_train_nnn.isnull().any().any() #Бинго!

False

In [161]:
X_test_nnn.isnull().any().any()

False

#### Нормализуем данные

Выделим колонки в отдельный список

In [124]:
headers = list (X_train_nnn.columns.values)

In [125]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_nnn_scaled = scaler.fit_transform(X_train_nnn)

In [162]:
X_test_nnn_scaled = scaler.fit_transform(X_test_nnn)

### Случайный лес и важность признаков

In [121]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(n_estimators=10, max_depth=5, min_samples_leaf=20, max_features=0.5, n_jobs=-1)
clf_rf.fit(X_train_nnn_scaled, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features=0.5,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [123]:
imp = pd.Series(clf_rf.feature_importances_)
imp.sort_values(ascending=False)

14    0.135392
11    0.110100
1     0.106092
9     0.076972
4     0.076874
10    0.067507
12    0.052806
2     0.044463
7     0.043142
25    0.039623
0     0.036172
17    0.032883
21    0.021716
23    0.020449
6     0.020090
26    0.019061
28    0.019016
5     0.015228
30    0.014508
8     0.012087
22    0.008814
16    0.007894
24    0.007266
27    0.006383
19    0.005463
13    0.000000
15    0.000000
32    0.000000
18    0.000000
20    0.000000
3     0.000000
29    0.000000
31    0.000000
33    0.000000
dtype: float64

In [133]:
print('Cамые важные признаки:')
headers[14], headers[11], headers[1]

Cамые важные признаки:


('GrLivArea', '1stFlrSF', 'LotArea')

### Стекинг моделей

In [147]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [148]:
classifier = StackingClassifier(
    [
        ('kn', KNeighborsClassifier()),
        ('lr', LogisticRegression()),
        ('dt', DecisionTreeClassifier())
    ],
LogisticRegression())

In [149]:
classifier.fit(X_train_nnn_scaled, y_train)

StackingClassifier(cv=None,
                   estimators=[('kn',
                                KNeighborsClassifier(algorithm='auto',
                                                     leaf_size=30,
                                                     metric='minkowski',
                                                     metric_params=None,
                                                     n_jobs=None, n_neighbors=5,
                                                     p=2, weights='uniform')),
                               ('lr',
                                LogisticRegression(C=1.0, class_weight=None,
                                                   dual=False,
                                                   fit_intercept=True,
                                                   intercept_scaling=1,
                                                   l1_ratio=None, max_iter=100,
                                                   multi_class='auto',
                  

In [150]:
classifier.named_estimators_['lr']

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [164]:
y_pred_proba_kn = classifier.named_estimators_['kn'].predict_proba(X_test_nnn_scaled)

In [165]:
y_pred_proba_lr = classifier.named_estimators_['lr'].predict_proba(X_test_nnn_scaled)

In [166]:
y_pred_proba_dt = classifier.named_estimators_['dt'].predict_proba(X_test_nnn_scaled)

In [163]:
y_pred_proba = classifier.predict_proba(X_test_nnn_scaled)

In [170]:
y_pred_proba_kn

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [172]:
y_pred_proba_lr

array([[1.82753792e-07, 9.35757782e-08, 2.83101913e-08, ...,
        2.06242636e-02, 3.01733103e-04, 4.97579076e-03],
       [3.94742921e-05, 2.97068967e-05, 6.91692942e-05, ...,
        1.42221331e-05, 1.10296678e-05, 1.57754370e-05],
       [2.87812770e-04, 2.39502683e-03, 8.43405889e-04, ...,
        1.91792091e-06, 7.11895178e-06, 1.59566496e-06],
       ...,
       [4.29919245e-07, 2.28513257e-08, 4.67299166e-08, ...,
        4.58103265e-04, 2.69088830e-04, 1.05064220e-03],
       [3.00266001e-05, 7.06090555e-12, 3.89960671e-10, ...,
        2.83675541e-08, 1.55360600e-07, 2.36077601e-07],
       [5.70326141e-08, 2.67501849e-08, 4.52381542e-09, ...,
        8.08122266e-06, 1.83574137e-06, 5.00795729e-04]])

In [173]:
y_pred_proba_dt

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [171]:
y_pred_proba

array([[0.00086163, 0.00086402, 0.00086798, ..., 0.00097849, 0.00090569,
        0.00093192],
       [0.00084791, 0.0008532 , 0.00085567, ..., 0.00085252, 0.00084945,
        0.00085483],
       [0.00085068, 0.00089141, 0.00085949, ..., 0.00085463, 0.00085181,
        0.00085675],
       ...,
       [0.00085832, 0.00086074, 0.00086467, ..., 0.00090498, 0.00087289,
        0.0008713 ],
       [0.0008556 , 0.00085796, 0.00086188, ..., 0.00086184, 0.00089356,
        0.00086338],
       [0.00086193, 0.00086436, 0.00086844, ..., 0.00086812, 0.00086427,
        0.00089333]])

In [174]:
classifier.final_estimator_

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)